# 新段落

In [9]:

#tensorflow_version 2.x
import os
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, Convolution2D, Flatten, MaxPool2D, Dropout, SpatialDropout2D, BatchNormalization, Activation

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

(mnist_imgs, mnist_lbls), (mnist_test_imgs, mnist_test_lbls) = tf.keras.datasets.mnist.load_data()
mnist_imgs = mnist_imgs.reshape(-1, 28, 28, 1) / 255.0
mnist_test_imgs = mnist_test_imgs.reshape(-1, 28, 28, 1) / 255.0




In [10]:
def create_model():
    net = Sequential([
    Input([28,28,1]),
    Convolution2D(96,(3,3),padding = "same"),
    Activation("relu"),
    Convolution2D(96,(3,3)),
    Activation("relu"),
    #
    MaxPool2D(pool_size=(2,2)),
    SpatialDropout2D(0.1),
    Convolution2D(192,(3,3),padding = "same"),
    Activation("relu"),
    Convolution2D(192,(3,3)),
    Activation("relu"),
    #
    MaxPool2D(pool_size=(2,2)),
    SpatialDropout2D(0.1),
    Flatten(),
    Dense(100),
    Activation("relu"),
    Dropout(0.1),
    #
    Dense(10,activation="softmax")
])

    net.compile(optimizer=tf.keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return net

In [11]:


net=create_model()
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建一个检查点回调
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
net.fit(mnist_imgs, mnist_lbls, batch_size=256, epochs=1,validation_split = 0.1,
       callbacks = [cp_callback])
#net.fit(fashion_imgs, fashion_lbls, batch_size=256, epochs=10,validation_split = 0.1)
#net.fit(cifar_imgs, cifar_lbls, batch_size=256, epochs=5,validation_split = 0.1)

Train on 54000 samples, validate on 6000 samples
53760/54000 [============================>.] - ETA: 1s - loss: 0.2198 - accuracy: 0.9324
Epoch 00001: saving model to training_1/cp.ckpt
54000/54000 [==============================] - 333s 6ms/sample - loss: 0.2190 - accuracy: 0.9327 - val_loss: 0.0486 - val_accuracy: 0.9873


How does net look like?

In [11]:
net.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 96)        960       
_________________________________________________________________
activation_5 (Activation)    (None, 28, 28, 96)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 26, 26, 96)        83040     
_________________________________________________________________
activation_6 (Activation)    (None, 26, 26, 96)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 96)        0         
_________________________________________________________________
spatial_dropout2d_2 (Spatial (None, 13, 13, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 13, 13, 192)      

In [12]:
test_result = net.evaluate(mnist_test_imgs,mnist_test_lbls,1000)
print(test_result)

10000/10000 [==============================] - 12s 1ms/sample - loss: 0.0444 - accuracy: 0.9868
[0.04439221452921629, 0.9868]


In [13]:
import numpy as np
net = create_model()
net.load_weights(checkpoint_path)
all_weights_of_whole_CNN = np.array(net.get_weights())       
#net.set_weights(a + 0.01)                 
#net.predict(mnist_test_imgs[0:10])

In [14]:
print(all_weights_of_whole_CNN[-4:-3]) 

[array([[ 0.01266974,  0.00676809, -0.00048612, ...,  0.01167931,
        -0.0383155 ,  0.01509606],
       [ 0.00864687, -0.02382355, -0.00308019, ..., -0.02987268,
        -0.02724801, -0.00793362],
       [-0.0256852 , -0.03435234,  0.01557891, ...,  0.00174566,
         0.00119009,  0.02896742],
       ...,
       [ 0.00537608, -0.00486933, -0.0064533 , ..., -0.03038361,
         0.03887499, -0.00468508],
       [ 0.0072107 , -0.01468464,  0.03506203, ...,  0.00627995,
        -0.0270571 , -0.01976817],
       [ 0.02022617, -0.03406528, -0.00191255, ..., -0.01203801,
        -0.02215942, -0.02948261]], dtype=float32)]


In [15]:
flatten1_to_dens2_weights_in2d = all_weights_of_whole_CNN[-4:-3][0].copy() 
#flatten1->dens2 480000 para
#print(flatten1_to_dens2_weights_in2d)

In [ ]:
flatten1_to_dens2_weights_in2d = all_weights_of_whole_CNN[-4:-3][0].copy() 
#flatten1->dens2 480000 para to be pruned

pruning_rate = 0.91
#hyper para, the ratio of weights set to 0(negelect, in other words)

flatten1_to_dens2_weights_in1d=flatten1_to_dens2_weights_in2d.flatten()   
#flatten those 2-d weights paras


Change_a_little_in2d_temp = flatten1_to_dens2_weights_in2d.copy()

for i in range(0, 4799):

  index_of_Descending_Order = np.argsort(-abs(Change_a_little_in2d_temp[i]))
  #use argsort(-1 * arg) to return the index of Descending Order of the weight value

  for j in range(0, 100):
    if (j >= int(99*(1-pruning_rate))):
      Change_a_little_in2d_temp[i][index_of_Descending_Order[j]] = 0
   
  
all_weights_of_whole_CNN[-4:-3][0] = Change_a_little_in2d_temp
net.set_weights(all_weights_of_whole_CNN)
test_result = net.evaluate(mnist_test_imgs,mnist_test_lbls,100)
# test_result = net.evaluate(fashion_val_imgs,fashion_val_lbls,100)
print(test_result)

# all_weights_of_whole_CNN[-4:-3][0] = flatten1_to_dens2_weights_in2d
# net.set_weights(all_weights_of_whole_CNN)
# test_result_original = net.evaluate(mnist_imgs,mnist_lbls,100)
# # test_result_original = net.evaluate(fashion_val_imgs,fashion_val_lbls,100)
# print(test_result_original)


In [17]:
net.summary()


[ 0.01266974  0.00676809 -0.00048612 ... -0.01203801 -0.02215942
 -0.02948261]
10000/10000 [==============================] - 12s 1ms/sample - loss: 0.1670 - accuracy: 0.9854
[0.1669769985228777, 0.9854]


In [22]:
net.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 96)        960       
_________________________________________________________________
activation_5 (Activation)    (None, 28, 28, 96)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 26, 26, 96)        83040     
_________________________________________________________________
activation_6 (Activation)    (None, 26, 26, 96)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 96)        0         
_________________________________________________________________
spatial_dropout2d_2 (Spatial (None, 13, 13, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 13, 13, 192)      